In [8]:
import pandas as pd

#need to cross validate...
#instead of 600 users try 1000 users and see if the model improves with more data
#use the full set of keywords instead from the big file rater tha the truncated one
#it may also use the credits file to find more features (human cast)
#note genres are the same in both data sets
#the data should be cleaned and added into a new set of csv files
#this set must be small enough to fit in a prject upload
#overview from the big dataset should also be included
#is average vote from 'newdata/movies_metadata.csv' the same as user ratings of that movie (maybe both can be used)
#production companies

#corpuse featurs (to combine for cossine similairity)
#genres, overview, production_companies, tagline, cast, crew, keywords
#quantitative: vote_average, rating, user rating average...


moviesFull1 = pd.read_csv('dataset/movie_dataset.csv')
moviesFull2 = pd.read_csv('newdata/movies_metadata.csv')
keywords = pd.read_csv('newdata/keywords.csv')
credits = pd.read_csv('newdata/credits.csv')


index =3

f = open("text.txt", "a")
f.write(moviesFull1.loc[index]['keywords']+"\n")
f.write(moviesFull1.loc[index]['cast']+"\n")
f.write(moviesFull1.loc[index]['genres']+"\n")
f.write(moviesFull1.loc[index]['director']+"\n")


f.write(str(list(moviesFull2[moviesFull2["title"] == str(moviesFull1.loc[index]["title"])]["genres"]))+ "\n")

f.write(str(list(keywords[keywords["id"] == int(moviesFull2[moviesFull2["title"] == str(moviesFull1.loc[index]["title"])]["id"])]
                     ["keywords"]))+ "\n")

f.write(str(credits.columns+ "\n"))

f.close()

print(moviesFull1["vote_average"])
print(moviesFull1["production_companies"])


#note: both movie metadata sets have genres and they are the same
#but the bigger set does not have keywords cast director
#instead keywords can be found in keywords data set
#cast can be found in credits dataset and the
#director can be found within credit of the credit data set



C:\Users\jackson\AppData\Local\Temp\ipykernel_25764\135711384.py:14: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  moviesFull2 = pd.read_csv('newdata/movies_metadata.csv')


0       7.2
1       6.9
2       6.3
3       7.6
4       6.1
       ... 
4798    6.6
4799    5.9
4800    7.0
4801    5.7
4802    6.3
Name: vote_average, Length: 4803, dtype: float64
0       [{"name": "Ingenious Film Partners", "id": 289...
1       [{"name": "Walt Disney Pictures", "id": 2}, {"...
2       [{"name": "Columbia Pictures", "id": 5}, {"nam...
3       [{"name": "Legendary Pictures", "id": 923}, {"...
4             [{"name": "Walt Disney Pictures", "id": 2}]
                              ...                        
4798             [{"name": "Columbia Pictures", "id": 5}]
4799                                                   []
4800    [{"name": "Front Street Pictures", "id": 3958}...
4801                                                   []
4802    [{"name": "rusty bear entertainment", "id": 87...
Name: production_companies, Length: 4803, dtype: object
